In [ ]:
from frechet_fda.data_generation_helpers import (
    gen_params_scenario_one,
    make_truncnorm_pdf,
)
from frechet_fda.distribution_tools import (
    get_optimal_range,
    make_distribution_objects,
    mean_func,
    pdf_to_qdf,
    qdf_to_pdf,
)

In [ ]:
# Set parameters
n = 200
grid_size = 10000
trunc = 3
mus, sigmas = gen_params_scenario_one(n)

In [ ]:
# Generate pdfs within truncation points
pdfs = make_truncnorm_pdf(-trunc, trunc, mus, sigmas, grid_size=grid_size)

In [ ]:
# Make Distribution class objects
my_pdfs = make_distribution_objects(pdfs)

In [ ]:
# For numerical correction: shorten the range for smaller sigmas to get rid of
# numerical artifacts when computing aintegrals, derivatives and means later
new_ranges = get_optimal_range(my_pdfs)

In [ ]:
# Generate pdfs again, this time within individual ranges
pdfs2 = [
    make_truncnorm_pdf(
        new_ranges[i][0], new_ranges[i][1], mus[i], sigmas[i], grid_size=grid_size,
    )[0]
    for i in range(n)
]

In [ ]:
# Generate all the distribution objects
new_pdfs = make_distribution_objects(pdfs2)
new_cdfs = [pdf.integrate() for pdf in new_pdfs]
new_qfs = [cdf.invert() for cdf in new_cdfs]
new_qdfs = [qf.differentiate() for qf in new_qfs]

In [ ]:
# Compare the new pdf with shorter range against the one with the initial range
min_sigma = sigmas.argmin()
max_sigma = sigmas.argmax()
my_pdfs[min_sigma].compare(new_pdfs[min_sigma] + 0.01)
my_pdfs[max_sigma].compare(new_pdfs[max_sigma] + 0.001)

In [ ]:
# New qdfs dont have astronomical numbers near the bounds
new_qdfs[min_sigma].y[0], new_qdfs[min_sigma].y[-1]

In [ ]:
# Compare different methods to compute mean qdf: either sequential transformations, as
# in new_qdfs, or formula 1/pdf(qf(t)), as implemented in pdf_to_qf func
mean_qdf = mean_func(new_qdfs)
mean_qdf2 = mean_func([pdf_to_qdf(pdf) for pdf in new_pdfs])
mean_qdf.compare(
    mean_qdf2 + 10,
    label_self="Sequential Transformations",
    label_other="Direct formula",
)

In [ ]:
# Compute Fréchet means
mean_pdf = mean_qdf.integrate().vcenter().invert().differentiate()
mean_pdf2 = qdf_to_pdf(mean_qdf2)
mean_pdf.compare(
    mean_pdf2 + 0.001,
    label_self="Sequential Transformations",
    label_other="Direct formula",
)

In [ ]:
temp = mean_pdf
temp = temp.integrate()
temp = temp.invert()
temp = temp.differentiate()
temp = temp.integrate()
temp.vcenter().plot()